## Vision transformers

Outline:
- Fill in model architecture (maybe ask them to implement attention by hand? might be too hard)
- Load pretrained weights
- Load ImageNet
- Fine-tune on ImageNet (write training loop)

## Vision transformer architecture

First, go to **vit_architecture.py** and fill in some model definitions. You might find this illustration of the architecture of a vision transformer helpful (Figure 1 from the [ViT paper](https://arxiv.org/pdf/2010.11929.pdf)).

![](vit_architecture.png)

Once you complete all the TODOs, run the cell below to import the model.

In [11]:
%load_ext autoreload
%autoreload 2

from vit_architecture_solutions import VisionTransformer
# from vit_architecture import VisionTransformer

config = {
    "num_classes": 10,
    "patch_size": 16,
    "hidden_size": 768,
    "model_name": 'ViT-S_16',
}

transformer_config = {
    "mlp_dim": 3072,
    "num_heads": 12,
    "num_layers": 12,
    "attention_dropout_rate": 0.0,
    "dropout_rate": 0.0,
}

model = VisionTransformer(
    num_classes=config["num_classes"], 
    patch_size=config["patch_size"],
    hidden_size=config["hidden_size"],
    model_name=config["model_name"],
    transformer_config=transformer_config,
)

model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


VisionTransformer(
    # attributes
    num_classes = 10
    patch_size = 16
    hidden_size = 768
    transformer_config = {'mlp_dim': 3072, 'num_heads': 12, 'num_layers': 12, 'attention_dropout_rate': 0.0, 'dropout_rate': 0.0}
    cls_head_bias_init = 0.0
    model_name = 'ViT-S_16'
)

## Load pre-trained weights

In [2]:
import os
import numpy as np
import jax
import jax.numpy as jnp
from vit_utils import load_pretrained
jax.local_devices() # shows available devices

2022-11-30 03:31:54.432863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 03:31:54.432935: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-30 03:31:54.432945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


[StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]

In [27]:
# Download pretrained S_16 checkpoint
checkpoint_file = "S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz"
if not os.path.exists("checkpoints"):
    os.system("mkdir checkpoints")
os.chdir("checkpoints")
if not os.path.exists(checkpoint_file):
    os.system(f"wget https://storage.googleapis.com/vit_models/augreg/{checkpoint_file}")
os.chdir("..")
with open(f"checkpoints/{checkpoint_file}", "rb") as f:
    ckpt_dict = np.load(f, allow_pickle=False)
    keys, values = zip(*list(ckpt_dict.items()))

# Initialize the model with random parameters
variables = jax.jit(lambda: model.init(
    jax.random.PRNGKey(0),
    # Discard the "num_local_devices" dimension of the batch for initialization.
    #batch['image'][0, :1],
    np.random.randn(2, 224, 224, 3),
    train=False,
), backend='cpu')()

In [34]:
# Load checkpoint
params = load_pretrained(
    pretrained_path=f'checkpoints/{checkpoint_file}',
    init_params=variables['params'],
)

load_pretrained: resized variant: %s to %s (1, 197, 384) (1, 197, 768)
load_pretrained: grid-size from %s to %s 14 14


## Load ImageNet

## Fine-tune